In [1]:
import os
os.chdir('../movies')
from movieLens import MovieLens

# Load the movie Lens class
ml = MovieLens()

# Algorithm

In [2]:
from surprise import Dataset, Reader, accuracy, AlgoBase, PredictionImpossible, NormalPredictor
from surprise.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
# Load the ratings dataset
ratings = ml.ratings.copy()
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
print(ratings['rating'].nunique())
list(ratings['rating'].unique())

10


[4.0, 5.0, 3.0, 2.0, 1.0, 4.5, 3.5, 2.5, 0.5, 1.5]

In [5]:
# Method from the Surprise library to load the DataFrame 
# Define the Reader object to parse the dataframe
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))

# Load the dataframe as a ratings dataset
ratingsDataset = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Build the full trainset
trainSet, testSet = train_test_split(ratingsDataset, test_size=0.2, random_state=42)
antitest = trainSet.build_anti_testset()

Para preparar los datos necesitamos crear conjuntos en formato de matriz con cada fila representando a un usuario y cada celda de la fila representando la valoración de cada película, dado que esta es la entrada esperada por el algoritmo RBM.

Para ello, necesitamos el número total de usuarios como números de fila y el número total de películas como número de columna
* num_users = trainSet.n_users
* num_movies = trainSet.n_items

In [6]:
def fitData(trainset):

    num_users = trainset.n_users
    num_movies = trainset.n_items

    # 3D matrix: users, movies and ratings
    # Ratings has size 10 given the possible rating values
    trainingMatrix = np.zeros([num_users, num_movies, 10], dtype=np.float32)

    for (uid, iid, rating) in trainset.all_ratings():
        adjustedRating = int(float(rating)*2.0) - 1
        trainingMatrix[int(uid), int(iid), adjustedRating] = 1

    # Flatten to a 2D array, with nodes for each possible rating type on each possible item, for every user.
    trainingMatrix = np.reshape(trainingMatrix, [trainingMatrix.shape[0], -1])

    return trainingMatrix

In [7]:
trainingMatrix = fitData(trainSet)
trainingMatrix.shape

(610, 89280)

In [8]:
"""
Updated on Sun Dec 1 08:32:13 2019

@author: Frank
@modified: Saurabh

@modified: Inés Gómez Fortis
"""

class RBM(AlgoBase):

    def __init__(self, visibleDimensions, epochs=20, hiddenDimensions=50, ratingValues=10, learningRate=0.001, batchSize=100):
        AlgoBase.__init__(self)
        self.visibleDimensions = visibleDimensions
        self.epochs = epochs
        self.hiddenDimensions = hiddenDimensions
        self.ratingValues = ratingValues
        self.learningRate = learningRate
        self.batchSize = batchSize

    def fit(self, trainset, trainingMatrix):
        
        AlgoBase.fit(self, trainset)
        self.Train(trainingMatrix)
        
        num_users = trainset.n_users
        num_movies = trainset.n_items
               
        self.predictedRatings = np.zeros([num_users, num_movies], dtype=np.float32)
        for uiid in range(trainset.n_users):
            if (uiid % 50 == 0):
                print("Processing user ", uiid)
            recs = self.GetRecommendations([trainingMatrix[uiid]])
            recs = np.reshape(recs, [num_movies, 10])
            
            for itemID, rec in enumerate(recs):
                # The obvious thing would be to just take the rating with the highest score: rating = rec.argmax()
                # but this just leads to a huge multi-way tie for 5-star predictions.
                # Instead, the paper suggests performing normalization over K values to get probabilities and take the expectation as the prediction
                normalized = np.exp(rec)/np.sum(np.exp(rec), axis=0)
                rating = np.average(np.arange(10), weights=normalized)
                self.predictedRatings[uiid, itemID] = (rating + 1) * 0.5
        
        return self        
        

    # in order to reconstract any users rating for any item            
    def Train(self, X):

        # Initialize weights randomly (earlier versions of thie code had this block inside MakeGraph, but that was a bug.)
        maxWeight = -4.0 * np.sqrt(6.0 / (self.hiddenDimensions + self.visibleDimensions))
        self.weights = tf.Variable(tf.random.uniform([self.visibleDimensions, self.hiddenDimensions], minval=-maxWeight, maxval=maxWeight), tf.float32, name="weights")
        self.hiddenBias = tf.Variable(tf.zeros([self.hiddenDimensions], tf.float32, name="hiddenBias"))
        self.visibleBias = tf.Variable(tf.zeros([self.visibleDimensions], tf.float32, name="visibleBias"))

        for epoch in range(self.epochs):
            
            trX = np.array(X)
            for i in range(0, trX.shape[0], self.batchSize):
                epochX = trX[i:i+self.batchSize]
                self.MakeGraph(epochX)

            print("Trained epoch ", epoch)

    # get back rating predictions for a given user
    def GetRecommendations(self, inputUser):
        
        feed = self.MakeHidden(inputUser)
        rec = self.MakeVisible(feed)
        return rec[0]       

    def MakeGraph(self, inputUser):
        
        # Perform Gibbs Sampling for Contrastive Divergence, per the paper we assume k=1 instead of iterating over the 
        # forward pass multiple times since it seems to work just fine
        
        # Forward pass
        # Sample hidden layer given visible...
        # Get tensor of hidden probabilities
        hProb0 = tf.nn.sigmoid(tf.matmul(inputUser, self.weights) + self.hiddenBias)
        # Sample from all of the distributions
        hSample = tf.nn.relu(tf.sign(hProb0 - tf.random.uniform(tf.shape(hProb0))))
        # Stitch it together
        forward = tf.matmul(tf.transpose(inputUser), hSample)
        
        # Backward pass
        # Reconstruct visible layer given hidden layer sample
        v = tf.matmul(hSample, tf.transpose(self.weights)) + self.visibleBias
        
        # Build up our mask for missing ratings
        vMask = tf.sign(inputUser) # Make sure everything is 0 or 1
        # User + movie + indicator of movie been rated
        vMask3D = tf.reshape(vMask, [tf.shape(v)[0], -1, self.ratingValues]) # Reshape into arrays of individual ratings
        vMask3D = tf.reduce_max(vMask3D, axis=[2], keepdims=True) # Use reduce_max to either give us 1 for ratings that exist, and 0 for missing ratings
        
        # Extract rating vectors for each individual set of 10 rating binary values
        v = tf.reshape(v, [tf.shape(v)[0], -1, self.ratingValues])
        vProb = tf.nn.softmax(v * vMask3D) # Apply softmax activation function
        vProb = tf.reshape(vProb, [tf.shape(v)[0], -1]) # And shove them back into the flattened state. Reconstruction is done now.
        # vProb = reconstructed data

        # Stitch it together to define the backward pass and updated hidden biases
        hProb1 = tf.nn.sigmoid(tf.matmul(vProb, self.weights) + self.hiddenBias)
        backward = tf.matmul(tf.transpose(vProb), hProb1)
    
        # Now define what each epoch will do...
        # Run the forward and backward passes, and update the weights
        weightUpdate = self.weights.assign_add(self.learningRate * (forward - backward))
        # Update hidden bias, minimizing the divergence in the hidden nodes
        hiddenBiasUpdate = self.hiddenBias.assign_add(self.learningRate * tf.reduce_mean(hProb0 - hProb1, 0))
        # Update the visible bias, minimizng divergence in the visible results
        visibleBiasUpdate = self.visibleBias.assign_add(self.learningRate * tf.reduce_mean(inputUser - vProb, 0))

        self.update = [weightUpdate, hiddenBiasUpdate, visibleBiasUpdate]
        
    def MakeHidden(self, inputUser):
        hidden = tf.nn.sigmoid(tf.matmul(inputUser, self.weights) + self.hiddenBias)
        self.MakeGraph(inputUser)
        return hidden
    
    def MakeVisible(self, feed):
        visible = tf.nn.sigmoid(tf.matmul(feed, tf.transpose(self.weights)) + self.visibleBias)
        #self.MakeGraph(feed)
        return visible
    
    def estimate(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unkown.')
        
        rating = self.predictedRatings[u, i]
        
        if (rating < 0.001):
            raise PredictionImpossible('No valid prediction exists.')
            
        return rating


In [9]:
# Create an RBM with (num items * rating values) visible nodes
model = RBM(trainingMatrix.shape[1])

# Fit the RBM model on the training set
model.fit(trainSet, trainingMatrix)

Trained epoch  0
Trained epoch  1
Trained epoch  2
Trained epoch  3
Trained epoch  4
Trained epoch  5
Trained epoch  6
Trained epoch  7
Trained epoch  8
Trained epoch  9
Trained epoch  10
Trained epoch  11
Trained epoch  12
Trained epoch  13
Trained epoch  14
Trained epoch  15
Trained epoch  16
Trained epoch  17
Trained epoch  18
Trained epoch  19
Processing user  0
Processing user  50
Processing user  100
Processing user  150
Processing user  200
Processing user  250
Processing user  300
Processing user  350
Processing user  400
Processing user  450
Processing user  500
Processing user  550
Processing user  600


# Metrics

In [10]:
import os
os.chdir('../metrics')
from metrics import evaluationMetrics
em = evaluationMetrics()

In [11]:
# Get test and antitest predictions
predtest = model.test(testSet)
predantitest = model.test(antitest)

# Get top N recommended movies for each user based on estimated ratings
top_10_RBM = em.getTopN(predantitest, minimumRating = 3.5)

In [12]:
top_10_RBM

defaultdict(list, {})

In [13]:
# Get top N recommended movies for each user based on estimated ratings
top_10_RBM = em.getTopN(predantitest, minimumRating = 3.0)

In [14]:
top_10_RBM

defaultdict(list,
            {432: [(112552, 3.1880362),
              (57669, 3.182102),
              (913, 3.181967),
              (31658, 3.1780846),
              (1233, 3.1730561),
              (134130, 3.171695),
              (741, 3.16995),
              (148626, 3.1677868),
              (81845, 3.1677234),
              (1884, 3.165959)],
             288: [(112552, 3.1885579),
              (913, 3.1824954),
              (31658, 3.178547),
              (1233, 3.1736052),
              (84152, 3.1723652),
              (134130, 3.1722314),
              (741, 3.17043),
              (81845, 3.168238),
              (148626, 3.1681397),
              (1884, 3.1664577)],
             599: [(57669, 3.1827314),
              (84152, 3.1726549),
              (1884, 3.1669552),
              (60684, 3.1664913),
              (1244, 3.1660051),
              (99114, 3.1657648),
              (914, 3.1628423),
              (44555, 3.156347),
              (7451, 3.1563184),
 

## Métricas de precisión: RMSE y MAE

In [15]:
# RMSE
rmse = accuracy.rmse(predtest)

# MAE
mae = accuracy.mae(predtest)

RMSE: 1.1615
MAE:  0.9636


## Métricas de relevancia: Precision, Recall y NDCG

In [16]:
# Precision
precisions = em.getPrecision(predtest, k=10, threshold=3.5)

# Mean Average Precision
mapModel = np.mean(list(precisions.values()))

# Recall
recalls = em.getRecall(predtest, k=10, threshold=3.5)

# Mean Average Recall
marModel = np.mean(list(recalls.values()))

# Normalized discounted cummulative gain (NDCG)
ndcgs, mean_ndcg = em.getNDCG(predtest,10)

## Otras métricas de interés: Coverage, User Coverage y Novelty

In [17]:
# Coverage
coverage = em.getCoverage(top_10_RBM,trainSet.n_items,trainSet.all_users())

# User coverage
user_coverage = em.getUserCoverage(top_10_RBM, trainSet.n_users,4)

# Novelty
novelty = em.getNovelty(top_10_RBM,trainSet)

Por último creamos un dataframe con todas las métricas de evaluación asociadas al modelo

In [18]:
cols = ["Model","RMSE","MAE","MAP","MAR","Mean_NDCG","Coverage","User_Coverage","Novelty"]
metrics_data = []

# Append the results to the list of dictionaries
metrics_data.append({"Model": "RBM", "RMSE": rmse, "MAE": mae, "MAP": mapModel, "MAR": marModel,
                     "Mean_NDCG": mean_ndcg, "Coverage": coverage, "User_Coverage": user_coverage,
                     "Novelty": novelty})

# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_data, columns=cols)
metrics_df

,Model,RMSE,MAE,MAP,MAR,Mean_NDCG,Coverage,User_Coverage,Novelty
0,RBM,1.161505,0.963611,0.163041,0.007933,0.936829,0.002688,0.0,678.94082


In [19]:
# Add the results to the dataframe with the metrics of all models.
em.addToMetricsDataframe(metrics_df)

# Random Algorithm

Vamos a evaluar también un modelo Random, en concreto "NormalPredictor" para poder comparar sus resultados con el resto de modelos. NormalPredictor es un algoritmo simple en Surprise que predice calificaciones aleatoriamente basado en la distribución del conjunto de entrenamiento. Supone una distribución normal de las calificaciones y genera predicciones aleatorias en función de esa distribución.

In [20]:
# Create the model
Random = NormalPredictor()
Random.fit(trainSet)

In [21]:
# Get test and antitest predictions
predtest_random = Random.test(testSet)
predantitest_random = Random.test(antitest)

# Get top N recommended movies for each user based on estimated ratings
top_10_random = em.getTopN(predantitest_random,minimumRating = 3.5)

## Compute metrics

In [22]:
# Accuracy Metrics
rmse_random = accuracy.rmse(predtest_random)
mae_random = accuracy.mae(predtest_random)

# Relevance metrics
precisions_random = em.getPrecision(predtest_random, k=10, threshold=3.5)
mapModel_random = np.mean(list(precisions_random.values()))

recalls_random = em.getRecall(predtest_random, k=10, threshold=3.5)
marModel_random = np.mean(list(recalls_random.values()))

ndcgs_random, mean_ndcg_random = em.getNDCG(predtest_random,10)

# Other metrics
coverage_random = em.getCoverage(top_10_random,trainSet.n_items,trainSet.all_users())
user_coverage_random = em.getUserCoverage(top_10_random, trainSet.n_users,4)
novelty_random = em.getNovelty(top_10_random,trainSet)

RMSE: 1.4218
MAE:  1.1387


In [23]:
cols = ["Model","RMSE","MAE","MAP","MAR","Mean_NDCG","Coverage","User_Coverage","Novelty"]
metrics_data = []

# Append the results to the list of dictionaries
metrics_data.append({"Model": "random", "RMSE": rmse_random, "MAE": mae_random, "MAP": mapModel_random, "MAR": marModel_random,
                     "Mean_NDCG": mean_ndcg_random, "Coverage": coverage_random, "User_Coverage": user_coverage_random,
                     "Novelty": novelty_random})

# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_data, columns=cols)
metrics_df

,Model,RMSE,MAE,MAP,MAR,Mean_NDCG,Coverage,User_Coverage,Novelty
0,random,1.421819,1.138744,0.638261,0.296262,0.927902,0.030578,1.0,1822.03918


In [24]:
# Add the results to the dataframe with the metrics of all models.
em.addToMetricsDataframe(metrics_df)

Y guardamos las recomendaciones resultantes del modelo para los dos usuarios de referencias (userId = 1 y userId = 4)

In [25]:
# Obtain the desired recomendations for user 1
recs_user1_RBM = [ml.movieID_to_name[movie_id] for movie_id, _ in top_10_RBM[1]]
recs_user1_Random = [ml.movieID_to_name[movie_id] for movie_id, _ in top_10_random[1]]

em.addUserRecommendations("rec_user1.xlsx", "RBM", recs_user1_RBM)
em.addUserRecommendations("rec_user1.xlsx", "Random", recs_user1_Random)


# Obtain the desired recomendations for user 4
recs_user4_RBM = [ml.movieID_to_name[movie_id] for movie_id, _ in top_10_RBM[4]]
recs_user4_Random = [ml.movieID_to_name[movie_id] for movie_id, _ in top_10_random[4]]

em.addUserRecommendations("rec_user4.xlsx", "RBM", recs_user4_RBM)
em.addUserRecommendations("rec_user4.xlsx", "Random", recs_user4_Random)